In [1]:
import subprocess as sp
import numpy as np
import pandas as pd
from io import StringIO
import os
import re
import shutil

from utils import *

In [2]:
#global parameters
cudadir = "/usr/common/software/cuda/11.0.167"
homedir = os.path.dirname(os.getcwd())

In [3]:
#input and output dirs
datadirs = ["/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla"]
outputdir = "/global/cfs/cdirs/m1759/yswang/results/rnn1d-tf2-noxla"

# Functions

In [4]:
def transpose_frame(df_metrics):
    #Copy the profile frame to make sure not to overwrite it and potentially read it in again if we screwed it up
    selectkeys = ["ID", "Precision", "Cell", "Input Shape", "Batch Size", "Time Steps", "Features", "Hidden Size", "Pass", "Name"]
    resultkeys = ["Precision", "Cell", "Input Shape", "Batch Size", "Time Steps", "Features", "Hidden Size", "Pass", "Name"]
                                    
    #as metricdf use df_summary
    metricdf = df_metrics.copy()
    #metricdf.sort_values(by=selectkeys,inplace=True)
    #metricdf.reset_index(drop=True, inplace=True)

    #remove the calibration
    metricdf = metricdf[metricdf["Pass"] != "calibrate"]
    profiledf = pd.DataFrame(columns=selectkeys)
    
    ####### Get timing information
    ### CUDA Time
    # get cycles
    metricname = "CUDA Cycles"
    cyclesdf = metricdf.loc[(metricdf["Metric Name"]=="sm__cycles_elapsed") & (metricdf["Metric Type"]=="total"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # get rates
    metricname = "CUDA Rates"
    ratesdf = metricdf.loc[(metricdf["Metric Name"]=="sm__cycles_elapsed") & (metricdf["Metric Type"]=="rate"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # check consistency
    if not cyclesdf[['ID', 'Name']].equals(ratesdf[['ID', 'Name']]):
        raise ValueError("CUDA Time data not consistent")
    # adjust metric unit
    ratesdf.loc[ratesdf["Metric Unit"].str.contains("cycle/nsecond"), ["CUDA Rates"]] *= 1e9
    # manual merge and compute CUDA Time
    cyclesdf["CUDA Rates"] = list(ratesdf["CUDA Rates"])
    cyclesdf["CUDA Time"] = cyclesdf["CUDA Cycles"] / cyclesdf["CUDA Rates"]
    # merge with output
    profiledf = cyclesdf[selectkeys+['CUDA Time']].copy()
    
    ### Combine
    del profiledf['ID']
    del metricdf['ID']
    profiledf['Invocations'] = 1
    profiledf = profiledf.groupby(resultkeys).sum().reset_index()
        
    ####### Get number of FLOPs
    
    ### FMA FLOPs = number of FMA instructions x 2
    metricdf.loc[metricdf["Metric Name"].str.contains("fma"), ["Metric Value"]] *= 2
    
    ### FP32 FLOPs
    metrics = ['sm__sass_thread_inst_executed_op_fadd_pred_on',
               'sm__sass_thread_inst_executed_op_ffma_pred_on',
               'sm__sass_thread_inst_executed_op_fmul_pred_on']
    tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "FP32 FLOPs"})
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["FP32 FLOPs"]], on=resultkeys, how="inner")
    
    ### FP16 FLOPs
    metrics = ['sm__sass_thread_inst_executed_op_hadd_pred_on',
               'sm__sass_thread_inst_executed_op_hfma_pred_on',
               'sm__sass_thread_inst_executed_op_hmul_pred_on']
    tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "FP16 FLOPs"})
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["FP16 FLOPs"]], on=resultkeys, how="inner")
    
    ### TC FLOPs
    tmpdf = metricdf.loc[ metricdf["Metric Name"].str.contains("sm__inst_executed_pipe_tensor"), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "TC FLOPs"})
    tmpdf["TC FLOPs"] = 512 * tmpdf["TC FLOPs"]
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["TC FLOPs"]], on=resultkeys, how="inner")

    ### Total FLOPs
    profiledf["FLOPs"] = profiledf["FP32 FLOPs"] + profiledf["FP16 FLOPs"] + profiledf["TC FLOPs"] #+ metricdf["FP64 FLOPs"]
        
    ### FLOPs fractions
    #profiledf["FP64 FLOPs Fraction"] = profiledf["FP64 FLOPs"]/profiledf["FLOPs"]
    profiledf["FP32 FLOPs Fraction"] = profiledf["FP32 FLOPs"]/profiledf["FLOPs"]
    profiledf["FP16 FLOPs Fraction"] = profiledf["FP16 FLOPs"]/profiledf["FLOPs"]
    profiledf["TC FLOPs Fraction"]   = profiledf["TC FLOPs"]/profiledf["FLOPs"]
    profiledf = profiledf.fillna(0.)

    ####### Get number of bytes
    
    # adjust metric unit
    metricdf.loc[(metricdf["Metric Unit"]=="Kbyte"), ["Metric Value"]] *= 1e3
    metricdf.loc[(metricdf["Metric Unit"]=="Mbyte"), ["Metric Value"]] *= 1e6
    metricdf.loc[(metricdf["Metric Unit"]=="Gbyte"), ["Metric Value"]] *= 1e9
    
    ### L1 Bytes
    #project out
    l1df = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__t_bytes"), selectkeys+["Metric Value"] ].copy()
    l1df = l1df.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "L1 Bytes"})
    # merge
    profiledf = profiledf.merge(l1df[resultkeys+["L1 Bytes"]], on=resultkeys, how="inner")
    
    ### L2 Bytes
    #project out
    l2df = metricdf.loc[metricdf["Metric Name"].str.contains("lts__t_bytes"), selectkeys+["Metric Value"] ].copy()
    l2df = l2df.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "L2 Bytes"})
    # merge
    profiledf = profiledf.merge(l2df[resultkeys+["L2 Bytes"]], on=resultkeys, how="inner")
    
    ### DRAM Bytes
    #project out
    dramdf = metricdf.loc[metricdf["Metric Name"].str.contains("dram__bytes"), selectkeys+["Metric Value"] ].copy()
    dramdf = dramdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "DRAM Bytes"})
    # merge
    profiledf = profiledf.merge(dramdf[resultkeys+["DRAM Bytes"]], on=resultkeys, how="inner")
    
    ### Get performance
    profiledf["Performance GFlop/s"] = profiledf["FLOPs"] / (profiledf["CUDA Time"]*10**9)

    ### Get AI
    # L1
    profiledf["L1 AI"]        = profiledf["FLOPs"]      / profiledf["L1 Bytes"]
    profiledf["FP32 L1 AI"]   = profiledf["FP32 FLOPs"] / profiledf["L1 Bytes"]
    profiledf["FP16 L1 AI"]   = profiledf["FP16 FLOPs"] / profiledf["L1 Bytes"]
    profiledf["TC L1 AI"]     = profiledf["TC FLOPs"]   / profiledf["L1 Bytes"]
    # L2
    profiledf["L2 AI"]        = profiledf["FLOPs"]      / profiledf["L2 Bytes"]
    profiledf["FP32 L2 AI"]   = profiledf["FP32 FLOPs"] / profiledf["L2 Bytes"]
    profiledf["FP16 L2 AI"]   = profiledf["FP16 FLOPs"] / profiledf["L2 Bytes"]
    profiledf["TC L2 AI"]     = profiledf["TC FLOPs"]   / profiledf["L2 Bytes"]
    # DRAM
    profiledf["DRAM AI"]      = profiledf["FLOPs"]      / profiledf["DRAM Bytes"]
    profiledf["FP32 DRAM AI"] = profiledf["FP32 FLOPs"] / profiledf["DRAM Bytes"]
    profiledf["FP16 DRAM AI"] = profiledf["FP16 FLOPs"] / profiledf["DRAM Bytes"]
    profiledf["TC DRAM AI"]   = profiledf["TC FLOPs"]   / profiledf["DRAM Bytes"]
    
    ### Cleanup
    profiledf.sort_values(by=resultkeys).reset_index(drop=True, inplace=True)

    return profiledf

# Import Data

In [5]:
#get all the files
files = []
for datadir in datadirs:
    files += [ os.path.join(datadir,x) for x in os.listdir(datadir) if ((os.path.splitext(x)[-1] == ".ncu-rep"))]

#recs
records = []

#build feature list:
for path in files:
    
    #filename
    file = os.path.basename(path)
    
    #path
    path = os.path.dirname(path)
    
    #splitup
    splt = file.split(".")
    
    prefix = ".".join(splt[0:-1])
    
    #append to records
    records.append({"prefix": prefix, "file": os.path.join(path, file)})

#put in df
recorddf = pd.DataFrame(records).sort_values(["prefix"])
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#display(recorddf["prefix"])

In [6]:
#group by prefixes and files
all_prefixes = set([x.split(".pass")[0] for x in recorddf["prefix"]])
xla_list = set([x.split(".")[-1] for x in recorddf["prefix"]])
if (len(xla_list) != 1):
    raise RuntimeError("too many xla options.")
xla = xla_list.pop()

all_passes = set([x.split(".pass_")[1].replace(".pass_","") for x in recorddf["prefix"].unique()])
all_passes = set([x.split(".")[0] for x in all_passes])

#metrics
df_profiles = []

for pref in all_prefixes:
    
    #set empty lists
    df_times = []
    df_timeline = []
    df_summary = []
    
    #print prefix
    #print(pref)
    
    #loop over passes
    df_times = []
    df_metrics = []
    for pas in all_passes:
        if pas == 'calibrate':
            continue
        
        #project frame
        files = recorddf.loc[ recorddf["prefix"] == pref + ".pass_" + pas + "." + xla, "file" ].values
        
        #project the invididual files
        metricfile = [x for x in files if x.endswith(".ncu-rep")][0]
        print(metricfile)
            
        #get the parameters from the filename
        parameters = parse_filename_nsight(os.path.basename(metricfile))
        #print(parameters)
            
        #metrics
        metricdf = import_nsight_metric(metricfile, cuda_dir=cudadir)
        for key in parameters:
            metricdf[key] = parameters[key]

        #fuse read/write metrics together:
        unique_metrics = metricdf["Metric Name"].unique()
        
        unique_metrics = set([x.replace(".sum","").replace(".per_second","").replace(".avg","") for x in unique_metrics])
        #add the metric type
        metricdf["Metric Type"] = "total"
        #rate
        metricdf.loc[ metricdf[ "Metric Name" ].str.contains(".per_second"), "Metric Type" ] = "rate"
                
        for metric in unique_metrics:
            metricdf.loc[ metricdf[ "Metric Name"].str.startswith(metric), "Metric Name" ] = metric
        
        unique_units = metricdf["Metric Unit"].unique()
        print(unique_units)
        
        #append to DF:
        df_metrics.append(metricdf)
    
    metricdf = pd.concat(df_metrics)
    
    #compute the profile
    profiledf = transpose_frame(metricdf)
    df_profiles.append(profiledf)

#concat everything
profiledf = pd.concat(df_profiles)
profiledf.reset_index(drop=True, inplace=True)

/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_32.celltype_lstm.input_16x16x32.nneu_16.pass_forward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']
/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_32.celltype_lstm.input_16x16x32.nneu_16.pass_backward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']


/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_32.celltype_lstm.input_16x16x32.nneu_32.pass_forward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']
/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_32.celltype_lstm.input_16x16x32.nneu_32.pass_backward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']
/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_16.celltype_lstm.input_16x16x32.nneu_32.pass_forward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']
/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_16.celltype_lstm.input_16x16x32.nneu_32.pass_backward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']
/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_32.celltype_lstm.input_32x16x32.nneu_16.pass_forward.noxla.ncu-rep
['Kbyte' 'Mbyte' 'cycle' 'cycle/nsecond' 'inst' 'byte']
/global/cfs/cdirs/m1759/yswang/data/rnn1d-tf2-noxla/tf2.fp_32.celltype_lstm.input_32x16x

In [7]:
display(profiledf)

,Precision,Cell,Input Shape,Batch Size,Time Steps,Features,Hidden Size,Pass,Name,CUDA Time,...,FP16 L1 AI,TC L1 AI,L2 AI,FP32 L2 AI,FP16 L2 AI,TC L2 AI,DRAM AI,FP32 DRAM AI,FP16 DRAM AI,TC DRAM AI
0,FP32,lstm,16x16x32,16,16,32,16,backward,_ZN10tensorflow87_GLOBAL__N__63_tmpxft_000027e...,0.000104,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1,FP32,lstm,16x16x32,16,16,32,16,backward,_ZN10tensorflow93_GLOBAL__N__69_tmpxft_0000243...,0.000010,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
2,FP32,lstm,16x16x32,16,16,32,16,backward,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.000008,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
3,FP32,lstm,16x16x32,16,16,32,16,backward,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.000005,...,0.0,0.0,0.000292,0.000292,0.0,0.0,0.000386,0.000386,0.0,0.0
4,FP32,lstm,16x16x32,16,16,32,16,backward,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.000457,...,0.0,0.0,0.039607,0.039607,0.0,0.0,0.055490,0.055490,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,FP32,lstm,16x16x64,16,16,64,16,forward,"void tensorflow::BiasNHWCKernel<float>(int, fl...",0.000057,...,0.0,0.0,0.086486,0.086486,0.0,0.0,0.146705,0.146705,0.0,0.0
1163,FP32,lstm,16x16x64,16,16,64,16,forward,void tensorflow::functor::DiagPartGpuKernel<fl...,0.000003,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1164,FP32,lstm,16x16x64,16,16,64,16,forward,void tensorflow::functor::FillPhiloxRandomKern...,0.000010,...,0.0,0.0,7.564541,7.564541,0.0,0.0,87.781433,87.781433,0.0,0.0
1165,FP32,lstm,16x16x64,16,16,64,16,forward,void tensorflow::functor::FillPhiloxRandomKern...,0.000014,...,0.0,0.0,0.246976,0.246976,0.0,0.0,2.897377,2.897377,0.0,0.0


# Compute AI Results

In [8]:
#sum over all kernels
combinedselectkeys = ["Precision", "Cell", "Input Shape", "Batch Size", "Time Steps", "Features", "Hidden Size", "Pass"]

#copy profiledf
combineddf = profiledf.copy()

#sum up
combineddf = combineddf.groupby(by=combinedselectkeys).sum()


#the flop fractions need to be recomputed
combineddf["FP32 FLOPs Fraction"] = combineddf["FP32 FLOPs"] / combineddf["FLOPs"]
combineddf["FP16 FLOPs Fraction"] = combineddf["FP16 FLOPs"] / combineddf["FLOPs"]
combineddf["TC FLOPs Fraction"]   = combineddf["TC FLOPs"]   / combineddf["FLOPs"]

### Get performance
combineddf["Performance GFlop/s"]      = combineddf["FLOPs"]      / (combineddf["CUDA Time"]*10**9)


### Get AI
# L1
combineddf["L1 AI"]        = combineddf["FLOPs"]      / combineddf["L1 Bytes"]
combineddf["FP32 L1 AI"]   = combineddf["FP32 FLOPs"] / combineddf["L1 Bytes"]
combineddf["FP16 L1 AI"]   = combineddf["FP16 FLOPs"] / combineddf["L1 Bytes"]
combineddf["TC L1 AI"]     = combineddf["TC FLOPs"]   / combineddf["L1 Bytes"]
# L2
combineddf["L2 AI"]        = combineddf["FLOPs"]      / combineddf["L2 Bytes"]
combineddf["FP32 L2 AI"]   = combineddf["FP32 FLOPs"] / combineddf["L2 Bytes"]
combineddf["FP16 L2 AI"]   = combineddf["FP16 FLOPs"] / combineddf["L2 Bytes"]
combineddf["TC L2 AI"]     = combineddf["TC FLOPs"]   / combineddf["L2 Bytes"]
# DRAM
combineddf["DRAM AI"]      = combineddf["FLOPs"]      / combineddf["DRAM Bytes"]
combineddf["FP32 DRAM AI"] = combineddf["FP32 FLOPs"] / combineddf["DRAM Bytes"]
combineddf["FP16 DRAM AI"] = combineddf["FP16 FLOPs"] / combineddf["DRAM Bytes"]
combineddf["TC DRAM AI"]   = combineddf["TC FLOPs"]   / combineddf["DRAM Bytes"]

combineddf.sort_values(by=combinedselectkeys).reset_index(drop=True, inplace=True)

In [9]:
display(combineddf)

CUDA Time  \
Precision Cell Input Shape Batch Size Time Steps Features Hidden Size Pass                  
FP16      lstm 128x16x32   128        16         32       16          backward   0.002696   
                                                                      forward    0.001129   
               16x128x32   16         128        32       16          backward   0.018401   
                                                                      forward    0.006721   
               16x16x128   16         16         128      16          backward   0.002498   
                                                                      forward    0.001036   
               16x16x32    16         16         32       16          backward   0.002358   
                                                                      forward    0.000988   
                                                          32          backward   0.002498   
                                                                      forward    0.000965   
                                                          64          backward   0.002572   
                                                                      forward    0.000988   
                                                          128         backward   0.002730   
                                                                      forward    0.001042   
               16x16x64    16         16         64       16          backward   0.002414   
                                                                      forward    0.001025   
               16x32x32    16         32         32       16          backward   0.004683   
                                                                      forward    0.001773   
               16x64x32    16         64         32       16          backward   0.009075   
                                                                      forward    0.003386   
               32x16x32    32         16         32       16          backward   0.002598   
                                                                      forward    0.001110   
               64x16x32    64         16         32       16          backward   0.002608   
                                                                      forward    0.001131   
FP32      lstm 128x16x32   128        16         32       16          backward   0.002931   
                                                                      forward    0.001192   
               16x128x32   16         128        32       16          backward   0.018409   
                                                                      forward    0.006775   
               16x16x128   16         16         128      16          backward   0.002372   
                                                                      forward    0.001008   
               16x16x32    16         16         32       16          backward   0.002428   
                                                                      forward    0.000981   
                                                          32          backward   0.002582   
                                                                      forward    0.000980   
                                                          64          backward   0.002482   
                                                                      forward    0.001022   
                                                          128         backward   0.002609   
                                                                      forward    0.001151   
               16x16x64    16         16         64       16          backward   0.002349   
                                                                      forward    0.001048   
               16x32x32    16         32         32       16          backward   0.004713   
                                                                      forward    

In [10]:
display(combineddf[["L2 AI", "L1 AI"]])

L2 AI  \
Precision Cell Input Shape Batch Size Time Steps Features Hidden Size Pass                 
FP16      lstm 128x16x32   128        16         32       16          backward  2.507119   
                                                                      forward   1.424997   
               16x128x32   16         128        32       16          backward  2.434198   
                                                                      forward   0.945726   
               16x16x128   16         16         128      16          backward  2.033212   
                                                                      forward   1.940686   
               16x16x32    16         16         32       16          backward  2.470812   
                                                                      forward   1.868394   
                                                          32          backward  2.086040   
                                                                      forward   1.693816   
                                                          64          backward  2.439429   
                                                                      forward   1.660525   
                                                          128         backward  3.258382   
                                                                      forward   1.825903   
               16x16x64    16         16         64       16          backward  2.372631   
                                                                      forward   1.892749   
               16x32x32    16         32         32       16          backward  2.439928   
                                                                      forward   1.538324   
               16x64x32    16         64         32       16          backward  2.421900   
                                                                      forward   1.208859   
               32x16x32    32         16         32       16          backward  2.916414   
                                                                      forward   2.161395   
               64x16x32    64         16         32       16          backward  2.452125   
                                                                      forward   1.654256   
FP32      lstm 128x16x32   128        16         32       16          backward  1.267799   
                                                                      forward   1.044882   
               16x128x32   16         128        32       16          backward  1.199467   
                                                                      forward   0.757143   
               16x16x128   16         16         128      16          backward  1.648232   
                                                                      forward   1.660354   
               16x16x32    16         16         32       16          backward  1.703766   
                                                                      forward   1.722124   
                                                          32          backward  2.030503   
                                                                      forward   1.449814   
                                                          64          backward  1.752749   
                                                                      forward   1.260631   
                                                          128         backward  1.799027   
                                                                      forward   1.206991   
               16x16x64    16         16         64       16          backward  1.624392   
                                                                      forward   1.695472   
               16x32x32    16         32         32       16          backward  1.479171   
                                                                      forward   1.354100   
               16x64x32    16      

# Export Data

In [11]:
profiledf.to_csv(os.path.join(outputdir,"full_profile.csv"))
combineddf.to_csv(os.path.join(outputdir,"combined_profile.csv"))